In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime
from datetime import timedelta

In [2]:
df = pd.read_csv('preprocessed_sales.csv')
df.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
r = df.sort_values(by='InvoiceDate' , ascending=False).iloc[0].InvoiceDate 
r = pd.to_datetime(r)
# strptime(x, "%Y-%m-%d %H:%M:%S")
last_day = r + timedelta(days=1)
last_day

Timestamp('2010-12-10 20:01:00')

In [4]:
dff = df.copy()
df_customer_segments = dff.groupby('CustomerId')['InvoiceDate'].max()
df_customer_segments = pd.DataFrame(df_customer_segments)
df_customer_segments['Recency'] =  dff.groupby('CustomerId')['InvoiceDate'].apply((lambda x : last_day - pd.to_datetime(x.max())))
df_customer_segments['Recency'] = df_customer_segments['Recency'].dt.days
df_customer_segments.drop('InvoiceDate', axis=1 , inplace=True)
df_customer_segments.head()

,Recency
CustomerId,
12346.0,165
12347.0,3
12348.0,74
12349.0,43
12351.0,11


In [5]:
df_customer_segments['Frequency'] = dff.groupby('CustomerId')['InvoiceNumber'].nunique()
df_customer_segments.head()

,Recency,Frequency
CustomerId,,
12346.0,165,11
12347.0,3,2
12348.0,74,1
12349.0,43,3
12351.0,11,1


In [6]:
dff['tp'] = dff['Quantity'] * dff['UnitPrice']
df_customer_segments['MonetaryValue'] = dff.groupby('CustomerId')['tp'].sum()
df_customer_segments

,Recency,Frequency,MonetaryValue
CustomerId,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,619.37
18284.0,67,1,461.68
18285.0,296,1,427.00


In [7]:
def func (r):
    if r <= rq1:
        return '1'
    elif rq1 < r <= rq2:
        return '2'
    elif rq2 < r <= rq3:
        return '3'
    else:
        return '4'
rq1 = df_customer_segments['Recency'].quantile(0.25)
rq2 = df_customer_segments['Recency'].quantile(0.5)
rq3 = df_customer_segments['Recency'].quantile(0.75)
df_customer_segments['R_quartile'] = df_customer_segments['Recency'].transform(func)

rq1 = df_customer_segments['Frequency'].quantile(0.25)
rq2 = df_customer_segments['Frequency'].quantile(0.5)
rq3 = df_customer_segments['Frequency'].quantile(0.75)
df_customer_segments['F_quartile'] = df_customer_segments['Frequency'].transform(func)


rq1 = df_customer_segments['MonetaryValue'].quantile(0.25)
rq2 = df_customer_segments['MonetaryValue'].quantile(0.5)
rq3 = df_customer_segments['MonetaryValue'].quantile(0.75)
df_customer_segments['M_quartile'] = df_customer_segments['MonetaryValue'].transform(func)

df_customer_segments

# df_customer_segments['F_quartile'] = None # To-Do
# df_customer_segments['M_quartile'] = None # To-Do
# df_customer_segments.head()

,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile
CustomerId,,,,,,
12346.0,165,11,372.86,4,4,2
12347.0,3,2,1323.32,1,2,3
12348.0,74,1,222.16,3,1,1
12349.0,43,3,2671.14,2,3,4
12351.0,11,1,300.93,1,1,1
...,...,...,...,...,...,...
18283.0,18,6,619.37,1,4,2
18284.0,67,1,461.68,3,1,2
18285.0,296,1,427.00,4,1,2


In [8]:
df_customer_segments['RFM'] = df_customer_segments['R_quartile'] + df_customer_segments['F_quartile'] + df_customer_segments['M_quartile']
df_customer_segments.head()

,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile,RFM
CustomerId,,,,,,,
12346.0,165,11,372.86,4,4,2,442
12347.0,3,2,1323.32,1,2,3,123
12348.0,74,1,222.16,3,1,1,311
12349.0,43,3,2671.14,2,3,4,234
12351.0,11,1,300.93,1,1,1,111


In [9]:
def func (val):
    if val == '144' :
        return "Best"
    elif val == "344" :
        return "AlmostLost"
    elif val == "444" :
        return "LostBigSpenders"
    elif val == "441" :
        return "LostCheap"
    elif val[1] == "4" :
        return "Loyal"
    elif val[2] == "4":
        return "BigSpenders"
    else:
        return "Normal"
df_customer_segments['Segment'] = df_customer_segments['RFM'].transform(func)
df_customer_segments=df_customer_segments.reset_index()
df_customer_segments.head()

,CustomerId,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile,RFM,Segment
0,12346.0,165,11,372.86,4,4,2,442,Loyal
1,12347.0,3,2,1323.32,1,2,3,123,Normal
2,12348.0,74,1,222.16,3,1,1,311,Normal
3,12349.0,43,3,2671.14,2,3,4,234,BigSpenders
4,12351.0,11,1,300.93,1,1,1,111,Normal
